# Dependencies

In [ ]:
!pip3 install pandas==2.0.3 openai==0.28.1 termcolor==2.3.0

# Imports

In [2]:
import json

import pandas as pd
import openai

from termcolor import colored

# Data Handling

In [3]:
def load_housing_data():
    return pd.read_csv("housing_data.csv")


def filter_lte(housing_data: pd.DataFrame,
               filter_name: str,
               filter_value: int):
    housing_data = housing_data[housing_data[filter_name] <= filter_value]
    return housing_data, len(housing_data)


def filter_gte(housing_data: pd.DataFrame,
               filter_name: str,
               filter_value: int):
    housing_data = housing_data[housing_data[filter_name] >= filter_value]
    return housing_data, len(housing_data)

In [4]:
housing_data = load_housing_data()
print(housing_data.dtypes)
housing_data.head()

price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


# OpenAI

## API Key

In [5]:
API_KEY_PATH = "../api_key"

try:
    openai.api_key = open(API_KEY_PATH, "r").read()
    print("API Key loaded")
except FileNotFoundError:
    print("Couldn't find the API key file")

API Key loaded


## Function Specifications

In [6]:
functions = [
    {
        "name": "filter_lte",
        "description": "Return the number of houses for which the specified filter is less than or equal to the filter value.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    },
    {
        "name": "filter_gte",
        "description": "Return the number of houses for which the specified filter is greater than or equal to the filter value.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    }
]

available_functions = {
    "filter_lte": filter_lte,
    "filter_gte": filter_gte
}

## API Calls

In [21]:
def call_completion_api(system_content: str, convo: list, model: str = "gpt-3.5-turbo"):

    completion = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": system_content}] + convo,
        functions=functions
    )

    return completion.choices[0]["message"]

def converse(system_content):
    convo = []
    housing_data = load_housing_data()
    system_content = system_content.format(len(housing_data))

    while True:

        # Get the user's input
        user_input = input()
        if user_input == "q":
            return convo

        # Add it to the convo
        convo.append(wrap_user_content(user_input))

        # Get ChatGPT response
        assistant_message = call_completion_api(system_content, convo)
        if assistant_message.get("function_call"):
            print("---")
            print(assistant_message.function_call)
            print("---")
            function_name = assistant_message["function_call"]["name"]
            function_to_call = available_functions[function_name]
            function_args = json.loads(assistant_message["function_call"]["arguments"])
            housing_data, expected_response = function_to_call(
                housing_data=housing_data,
                filter_name=function_args.get("filter_name"),
                filter_value=function_args.get("filter_value")
            )
            convo.append(assistant_message.to_dict_recursive())
            convo.append(wrap_function_content(function_name, expected_response))
            assistant_message = call_completion_api(system_content, convo)

        # Add response to convo
        convo.append(assistant_message.to_dict_recursive())

        # Display latest response
        print(colored(assistant_message.content, "blue"))


def wrap_user_content(input: str):
    return {"role": "user", "content": input}


def wrap_function_content(function_name: str, function_response):
    return {"role": "function", "name": function_name, "content": str(function_response)}

In [24]:
REALTOR_SYSTEM_CONTENT = \
"""
You are a helpful realtor with a list initially containing {} properties.
For each property, we have the following characteristics:
    price (asking price in USD),
    area (total floor area in metres squared),
    bedrooms (number of bedrooms),
    bathrooms (number of bathrooms),
    stories (number of stories),
    parking (number of parking spaces)
Help the user to narrow the list.
When there are fewer than 10, advise the user to call the office to arrange viewings.
"""

# I'm looking for a house. The budget is 5000000.

convo = converse(REALTOR_SYSTEM_CONTENT)

 I'm looking for a house. The budget is 5000000.


---
{
  "name": "filter_lte",
  "arguments": "{\n  \"filter_name\": \"price\",\n  \"filter_value\": 5000000\n}"
}
---
There are 350 houses within your budget of $5,000,000.


 We are a two car family.


---
{
  "name": "filter_gte",
  "arguments": "{\n  \"filter_name\": \"parking\",\n  \"filter_value\": 2\n}"
}
---
There are 54 houses that have at least 2 parking spaces, suitable for a two-car family.


 q


In [25]:
for m in convo:
    print(m)

{'role': 'user', 'content': "I'm looking for a house. The budget is 5000000."}
{'role': 'assistant', 'content': None, 'function_call': {'name': 'filter_lte', 'arguments': '{\n  "filter_name": "price",\n  "filter_value": 5000000\n}'}}
{'role': 'function', 'name': 'filter_lte', 'content': '350'}
{'role': 'assistant', 'content': 'There are 350 houses within your budget of $5,000,000.'}
{'role': 'user', 'content': 'We are a two car family.'}
{'role': 'assistant', 'content': None, 'function_call': {'name': 'filter_gte', 'arguments': '{\n  "filter_name": "parking",\n  "filter_value": 2\n}'}}
{'role': 'function', 'name': 'filter_gte', 'content': '54'}
{'role': 'assistant', 'content': 'There are 54 houses that have at least 2 parking spaces, suitable for a two-car family.'}
